In [25]:
from fastai import *
import pylab
import matplotlib
import gc
from glob import glob
import pandas as pd
import numpy as np
from pathlib import Path 

In [28]:
from fastai.vision.data import ImageDataLoaders

path = Path('/Users/srinijacherivirala/Desktop/all')
np.random.seed(42)
train_df = pd.read_csv('/Users/srinijacherivirala/Documents/github/noisedetection/train_labels.csv', index_col=0)
data = ImageDataLoaders.from_df(path = path, df = train_df, folder = 'all', valid_pct=0.2, suffix='.jpg', num_workers=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/srinijacherivirala/Desktop/all/all/5AEF6B2F'

In [13]:
data.classes

['Gunshot', 'Non_Gunshot']

In [14]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

/usr/local/lib/python3.9/dist-packages/fastai/vision/learner.py:106: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|████

In [ ]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-3))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

In [ ]:
learn.save('stage-3')

Making Predictions

In [ ]:
with open('output.csv',"w") as file:
    file.write("ID,Prediction\n")
    for file in labels_test.filename:
        img = open_image('test/'+str(file)+'.jpg')
        prediction = str(learn.predict(img)[0]).split()[0]
        file.write(str(file)+','+prediction)
        file.write('\n')

In [ ]:
output = pd.read_csv('output.csv')
output.head()

Cleanup

In [ ]:
'''
%%capture
!apt-get install zip
!zip -r train.zip /kaggle/working/train/
!zip -r test.zip /kaggle/working/test/
!rm -rf train/*
!rm -rf test/*
'''